In [2]:
import equinox as eqx
import jax
import jax.numpy as jnp

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [31]:
key = jax.random.PRNGKey(446)

class RNN(eqx.Module): 
    hidden_dim: int
    Wh: eqx.nn.Linear
    Wx: eqx.nn.Linear
    Wy: eqx.nn.Linear
    act: callable
    
    # TODO: check bias 
    def __init__(self, input_dim=2, hidden_dim=2, output_dim=1):
        self.hidden_dim = hidden_dim
        self.Wh = eqx.nn.Linear(hidden_dim, hidden_dim, key=key)
        self.Wx = eqx.nn.Linear(input_dim, hidden_dim, key=key)
        self.Wy = eqx.nn.Linear(hidden_dim, output_dim, key=key)
        self.act = jax.nn.sigmoid

    def __call__(self, x): 
        batch_size, seq_len, _ = x.shape

        # hidden = torch.nn.Parameter(torch.zeros(batch_size, self.hidden_dim))
        hidden = jnp.zeros((self.hidden_dim, batch_size))
        outs = []

        for i in range(seq_len):
            # print(self.Wh(hidden).shape)
            print(hidden.shape)
            hidden = self.act(self.Wh(hidden) + self.Wx(x[:,i,:]))
            out = self.Wy(hidden)
            outs.append(out)
        
        # output shape = (batch_size, sequence length, output dimension)
        # return torch.stack(outs).permute(1,0,2)
        stacked = jnp.stack(outs, axis=1)
        return jnp.transpose(stacked, (1,0,2))


In [32]:
model = RNN(input_dim=2, hidden_dim=2, output_dim=1)

batch_size = 128
seq_len = 10
input_dim = 2
x = jax.random.normal(key, (batch_size, seq_len, input_dim))
print(model)
model(x)

RNN(
  hidden_dim=2,
  Wh=Linear(
    weight=f32[2,2],
    bias=f32[2],
    in_features=2,
    out_features=2,
    use_bias=True
  ),
  Wx=Linear(
    weight=f32[2,2],
    bias=f32[2],
    in_features=2,
    out_features=2,
    use_bias=True
  ),
  Wy=Linear(
    weight=f32[1,2],
    bias=f32[1],
    in_features=2,
    out_features=1,
    use_bias=True
  ),
  act=<wrapped function sigmoid>
)
(128, 2)


TypeError: dot_general requires contracting dimensions to have the same shape, got (2,) and (128,).